In [ ]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date, datetime, timedelta
from pprint import pprint
import re
import time

In [169]:
def format_date(st):
    """
    Функция получает строку, выражающую дату, с сайта и преобразует ее к формату datetime
    """
    def conversion_date(st):
        """
        Функция конвертер, получает строку(название месяца), возвращает число соответствующее месяцу
        """
        mont = {
            'января':1,
            'февраля':2,
            'марта':3,
            'апреля':4,
            'мая':5,
            'июня':6,
            'июля':7,
            'августа':8,
            'сентября':9,
            'октября':10,
            'ноября':11,
            'декабря':12,
        } 

        try:
            return mont[st]
        except KeyError:
            return None
        
        
    def get_date(res):  
        """
        Функция проводит синтаксический разбор строки, которая представляет собой выражение даты, возвращает соответствующий datetime
        """
        if res[3] != "":           
            #преобразуем в дату ('25 ноября', '25', 'ноября', '2019', '14:09')
            date1 = date(int(res[3]), conversion_date(res[2]), int(res[1]))
            time = datetime.strptime(res[4], '%H:%M')
            date_time = datetime.combine(date1,time.time())
            #datetime(2017, 3, 25)
        elif res[1] != "":
            #преобразуем в дату ('4 марта', '4', 'марта', '', '16:05')

            date1 = datetime(date.today().year, conversion_date(res[2]), int(res[1]))
            time = datetime.strptime(res[4], '%H:%M')
            date_time = datetime.combine(date1,time.time())  

        elif res[2] != "" and res[2].upper() == "Сегодня".upper():
            #преобразуем в дату ('Сегодня', '', 'Сегодня', '', '12:40')

            date1 = date.today()
            time = datetime.strptime(res[4], '%H:%M')
            date_time = datetime.combine(date1,time.time())  

        elif res[2] != "" and res[2].upper() == "Вчера".upper():
            #преобразуем в дату ('Вчера', '', 'Вчера', '', '19:37')
            date1 = date.today() - timedelta(days=1)    
            time = datetime.strptime(res[4], '%H:%M')
            date_time = datetime.combine(date1,time.time()) 

        else:
            #преобразуем вернуть null    
            date_time = None
        return date_time
    
    res = re.findall(r'((\d{1,2})?\s*(\w+))\W+(\d{4})?\W+(\d{1,2}:\d{1,2})', st)
    res = res[0] if res else []
    return get_date(res)

        
def add_element_one(collection, obj, element):
    '''
    Функция добавляет информацию об одном письме
    '''
    if collection.count_documents({element:obj[element]}) == 0:
        collection.insert_one(obj)
    
    
def add_elements_base(collection, items_news:list, element):
    '''
    Функция добавляет информацию о нескольуих писем
    '''
    for item in items_news:
        add_element_one(collection, item, element)

### 1. Написать программу, которая собирает входящие письма из своего или тестового почтового ящика, и сложить информацию о письмах в базу данных (от кого, дата отправки, тема письма, текст письма).

In [ ]:
client = MongoClient('localhost', 27017)
db = client['mail']

In [ ]:
chrome_options = Options()
chrome_options.add_argument('start-maximized')
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://e.mail.ru/')
param = {'password':'555555555',
         'login':'d55555555555@mail.ru'}


button = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'button'))
        )

elem = driver.find_element_by_name("username")
elem.send_keys(param['login'])


button = driver.find_element(By.TAG_NAME, 'button')
button.click()
   
    
elem = WebDriverWait(driver,10).until(
            EC.visibility_of_element_located((By.NAME, 'password'))
        )    

elem.send_keys(param['password'])

button = driver.find_element(By.TAG_NAME, 'button')
button.click()



elem = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'js-letter-list-item'))
        )

total_elems = []

#ограничеваем выборку элементов до 200


while len(total_elems) < 200:
    elems = driver.find_elements_by_class_name('js-letter-list-item')
    count_total_elems = len(total_elems)
    for item in elems:
        #driver.execute_script("arguments[0].setAttribute('class','delete')", item)
        total_elems = total_elems + [item.get_attribute("href")]
    total_elems = list(set(total_elems))
    if count_total_elems == len(total_elems):
        break
    
    actions = ActionChains(driver)
    actions.move_to_element(elems[-1])  
    actions.perform()


pprint(f"количество элементов до  {len(total_elems)}  и после  {len(list(set(total_elems)))}")


elem = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'js-letter-list-item'))
        )
latters = []

for item in total_elems:
    driver.get(item)

    # Ожидаем загрузки страницы
    page_mail = WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'thread_direction-next')]"))
            )

    # Пролучаем данные письма

    letter_subject = page_mail.find_element(By.XPATH, ".//h2[contains(@class,'thread__subject')]").text
    from_whom_letter = page_mail.find_element(By.XPATH, ".//span[contains(@class,'letter-contact')]")
    from_whom_letter_email = from_whom_letter.get_attribute("title")
    from_whom_letter = from_whom_letter.text
    date_latter = format_date(page_mail.find_element(By.XPATH, ".//div[contains(@class,'letter__date')]").text)
    letter_body = page_mail.find_element(By.XPATH, ".//div[contains(@class,'letter-body')]").text
    obj = {"letter_subject":letter_subject,
           "from_whom_letter":from_whom_letter,
           "from_whom_letter_email":from_whom_letter_email,
           "date_latter":date_latter,
           "letter_body":letter_body,
           "link_latter":item
          }
    add_element_one(db.mail, obj, 'link_latter')
    latters = latters + [obj]
#pprint(latters)

In [ ]:
#db.mail.delete_many({})

In [ ]:
for item in db.mail.find({}):
        pprint(item)

### 2. Написать программу, которая собирает «Хиты продаж» с сайтов техники М.видео, ОНЛАЙН ТРЕЙД и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары. 

##### Мвидео

In [ ]:
client = MongoClient('localhost', 27017)
db = client['bestseller']

chrome_options = Options()
chrome_options.add_argument('start-maximized')
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.mvideo.ru/')

#//div[contains(@class,'sel-hits-block')][2]
#//div[contains(@class,'u-mb-0')][contains(normalize-space(text()),normalize-space('Хиты продаж'))]/ancestor::div[@class = 'section']


####### Выбераем регион
sity = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@href='#header-regions']"))
        )

ActionChains(driver).move_to_element(sity).click().perform()

input_sity = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.ID, 'region-selection-form-city-input'))
        )

input_sity.send_keys("Мурманск")

dutton_sity = driver.find_element(By.ID, 'region-selection-form-city-input-btn')
ActionChains(driver).move_to_element(dutton_sity).click().perform()
#######



section = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'u-mb-0')][contains(normalize-space(text()),normalize-space('Хиты продаж'))]/ancestor::div[@class = 'section']"))
        )

actions = ActionChains(driver)
actions.move_to_element(section)  
actions.perform()

#Прожимаем кнопку, чтоб получить все карточки товара
#
#
#


button_next = section.find_element(By.XPATH, ".//a[@class='next-btn sel-hits-button-next']")
actions = ActionChains(driver)
actions.move_to_element(section)
actions.perform()
count = 0
bestsellers = []
buff = []
while True :
    if not('disabled' in button_next.get_attribute("class")):  
        button_next1 = WebDriverWait(driver,15).until(
            EC.visibility_of(button_next)
        )
        ActionChains(driver).move_to_element(button_next).perform()
        
        ########## парсинг объекта
        blocks = section.find_elements(By.XPATH, ".//li[@class='gallery-list-item height-ready']")
        for item in blocks:            
            product = item.find_element(By.CLASS_NAME, 'sel-product-tile-title')
            product_link = product.get_attribute("href")
            if product_link in buff:
                break
            
            buff += [product_link]
            product_name = product.get_attribute("title")            
            price_current = item.find_elements(By.XPATH, ".//div[contains(@class,'c-pdp-price__current')]")
            price_current = re.sub(r'([\s|¤])', '', price_current[0].text) if price_current else None          
         
            obj = {'product_name':product_name,
                   'product_link':product_link,
                   'price_current':price_current,
                   'parsing_date' :datetime.today()
                  }
            bestsellers = bestsellers + [obj]
        #################
        
        button_next1.click()
        count+=1
       
    else:
        pprint(f"количество нажатий {count}")
        break

         
pprint(len(bestsellers))
bestsellers 

add_elements_base(db.mvideo, bestsellers, 'product_link')

In [ ]:
for item in db.mvideo.find({}):
        pprint(item)

#### onlinetrade.ru

In [ ]:
client = MongoClient('localhost', 27017)
db = client['bestseller']

chrome_options = Options()
chrome_options.add_argument('start-maximized')
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.onlinetrade.ru/')


block = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@id='tabs_hits']"))
        )

ActionChains(driver).move_to_element(block).perform()

button = block.find_element(By.XPATH, ".//span[@class='swiper-button-next ic__hasSet ic__hasSet__arrowNextBlue']")
buff = []
bestsellers = []

while block.find_elements(By.XPATH, ".//span[@class='swiper-button-next ic__hasSet ic__hasSet__arrowNextBlue']"):    
    items = block.find_elements(By.XPATH, ".//div[@class='indexGoods__item']")
    for item in items:
        product_name = item.find_elements(By.XPATH, ".//a[@class='indexGoods__item__name']")
        product_link = product_name[0].get_attribute("href") if product_name else None
        product_name = product_name[0].get_attribute("title") if product_name else None
        price_current = item.find_elements(By.XPATH, ".//span[contains(@class,'price')]")
        price_current_count = len(price_current)
        price_current = price_current[0].text if price_current else None

        if not(price_current) :#or not(price_current)
                    continue 
        
        if (product_link in buff) :#or not(price_current)
                    continue            
        buff += [product_link] 
        
        obj = { 'product_name':product_name,
                'product_link':product_link,
                'price_current':price_current,
                'parsing_date':datetime.today(),
                'price_current_count':price_current_count
              }
        bestsellers += [obj]
    #time.sleep(2) swiper-button-next ic__hasSet ic__hasSet__arrowNextBlue
    button.click()
    
    
add_elements_base(db.onlinetrade, bestsellers, 'product_link')

In [ ]:
for item in db.onlinetrade.find({}):
        pprint(item)

In [221]:
# pprint(len(items))
# pprint(len(bestsellers))
# pprint(bestsellers)

12
9
[{'parsing_date': datetime.datetime(2020, 7, 13, 15, 40, 41, 641496),
  'price_current': '1 790 ₽',
  'price_current_count': 1,
  'product_link': 'https://www.onlinetrade.ru/catalogue/kostochkovydavlivateli-c1060/leifheit/pribor_dlya_udaleniya_kostochek_u_vishni_leifheit_37200_cherrymat-72852.html',
  'product_name': 'Подробнее о «Прибор для удаления косточек у вишни LEIFHEIT '
                  '37200 CHERRYMAT»'},
 {'parsing_date': datetime.datetime(2020, 7, 13, 15, 40, 41, 679396),
  'price_current': '3 505 ₽',
  'price_current_count': 1,
  'product_link': 'https://www.onlinetrade.ru/catalogue/elektricheskie_zubnye_shchetki-c81/oral_b/zubnaya_shchetka_elektricheskaya_oral_b_d16.513u_professional_care_blue_500_d16.513u_blue-187688.html',
  'product_name': 'Подробнее о «Зубная щетка электрическая Oral-B D16.513U '
                  'Professional Care, blue»'},
 {'parsing_date': datetime.datetime(2020, 7, 13, 15, 40, 41, 726270),
  'price_current': '18 990 ₽',
  'price_current_cou

In [ ]:
w = [item for item in bestsellers if item['price_current'] != ""]
pprint(len(w))
pprint(w)